In [1]:
from src.models.representation.vae import *
from src.dataloader.dataset import MidiDataset
from src.dataloader.dataloader import MidiDataLoader



In [2]:
dataset = MidiDataset(
   midi_files_dir="data/midi_dataset/midis", frame_per_second= 32, verbose = False
)

In [3]:
from torch.utils.data import DataLoader
encoder = LSTMVaeEncoder(
    128, 
    256,
    128,
    4
)
decoder = LSTMVaeDecoder(
    128, 
    256,
    4,
    128
)

vae = RecurrentVae(encoder, decoder).cuda()


midi_dataloader = MidiDataLoader(
    dataset,
    batch_size=4,
    pin_memory=True,
    num_workers=4,
    shuffle=True,
)

(dataset[0] != 0).float().sum().item()


t = next(iter(midi_dataloader))


In [4]:
import torch

optimizer = torch.optim.Adam(
    vae.parameters(),
    lr=1e-3,
)

loss_fn = VaeLoss()

trainer = VaeTrainer(
    vae,
    optimizer,
    loss_fn,
    "test/simple_vae",
    100
)

trainer.train(midi_dataloader, epochs=10)
trainer.save("test/simple_vae_final.pth")

RuntimeError: cuDNN error: CUDNN_STATUS_NOT_SUPPORTED. This error may appear if you passed in a non-contiguous input.